# Tutorial about computing hulls for localization data

For each set of localizations with 2D or 3D spatial coordinates various hull can be computed. A hull can be the minimal bounding box, the oriented minimal bounding box, the convex hull, or an alpha shape. 

You can trigger computation of specific hull objects using a specific hull class or from the corresponding LocData attribute.

In [ ]:
from pathlib import Path

%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import locan as lc
from locan.data.hulls import BoundingBox, ConvexHull, OrientedBoundingBox

In [ ]:
lc.show_versions(system=False, dependencies=False, verbose=False)

## Synthetic data

In [ ]:
rng = np.random.default_rng(seed=1)

In [ ]:
locdata = lc.simulate_Thomas(parent_intensity=1e-3, region=((0, 100), (0, 100)), cluster_mu=10, cluster_std=2, seed=rng)

locdata.print_summary()

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
plt.show()

## Minimal bounding box for spatial coordinates

In [ ]:
# H = BoundingBox(locdata.coordinates)

H = locdata.bounding_box

print('dimension: ', H.dimension)
print('hull: ', H.hull)
print('width: ', H.width)
print('vertices: ', H.vertices)
print('region_measure: ', H.region_measure)
print('subregion_measure: ', H.subregion_measure)
print('region: ', H.region)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.add_patch(locdata.bounding_box.region.as_artist(alpha=0.2))
locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
plt.show()

## Oriented minimal bounding box for spatial coordinates

In [ ]:
# H = OrientedBoundingBox(locdata.coordinates)

H = locdata.oriented_bounding_box

print('dimension: ', H.dimension)
print('hull: ', H.hull)
print('vertices: ', H.vertices)
print('width: ', H.width)
print('region_measure: ', H.region_measure)
print('subregion_measure: ', H.subregion_measure)
print('region: ', H.region)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.add_patch(locdata.oriented_bounding_box.region.as_artist(alpha=0.2))
locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
plt.show()

## Convex hull

### Convex hull for spatial coordinates (scipy)

In [ ]:
# H = ConvexHull(locdata.coordinates, method='scipy')

H = locdata.convex_hull

print('dimension: ', H.dimension)
print('hull: ', H.hull)
# print('vertex_indices: ', H.vertex_indices)
# print('vertices: ', H.vertices)
print('region_measure: ', H.region_measure)
print('subregion_measure: ', H.subregion_measure)
print('points on boundary: ', H.points_on_boundary)
print('points on boundary relative to all points: ', H.points_on_boundary_rel)
print('region: ', H.region)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.add_patch(locdata.convex_hull.region.as_artist(alpha=0.2))
locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
plt.show()

### Convex hull for spatial coordinates (shapely)

Some hulls can be computed from different algorithms. If implemented, use the `methods` parameter to specify the algorithm.

In [ ]:
H = ConvexHull(locdata.coordinates, method='shapely')

print('dimension: ', H.dimension)
print('hull: ', H.hull)
# print('vertices: ', H.vertices)
print('region_measure: ', H.region_measure)
print('subregion_measure: ', H.subregion_measure)
print('points on boundary: ', H.points_on_boundary)
print('points on boundary relative to all points: ', H.points_on_boundary_rel)
print('region: ', H.region)

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.add_patch(H.region.as_artist(alpha=0.2))
locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
plt.show()

## Alpha shape for spatial coordinates

The alpha shape depends on a single parameter `alpha` (not to confuse with the alpha to specify opacity in figures). The alpha complex is an alpha-independent representation of all alpha shapes.

You can get all apha values for which the corresponding alpha shape changes.

In [ ]:
lc.AlphaComplex(locdata.coordinates).alphas()

You can determine an optimal `alpha`, i.e. the smallest `alpha` for which all points are still part of the alpha shape.

In [ ]:
opt_alpha = lc.AlphaComplex(locdata.coordinates).optimal_alpha()
opt_alpha

In [ ]:
# H = lc.AlphaShape(opt_alpha, locdata.coordinates)

locdata.update_alpha_shape(alpha=opt_alpha)
H = locdata.alpha_shape

print('dimension: ', H.dimension)
# print('vertex_indices: ', H.vertex_indices)
# print('vertices: ', H.vertices)
print('region_measure: ', H.region_measure)
print('subregion_measure: ', H.subregion_measure)
print('points in alpha shape: ', H.n_points_alpha_shape)
print('points in alpha shape relative to all points: ', H.n_points_alpha_shape_rel)
print('points on boundary: ', H.n_points_on_boundary)
print('points on boundary relative to all points: ', H.n_points_on_boundary_rel)
print('region: ', H.region)

The alpha shape is made of different vertex types that can be differentiated as *exterior*, *interior*, *regular* or *singular*.

In [ ]:
ac_simplices_all = H.alpha_complex.get_alpha_complex_lines(H.alpha, type='all')
ac_simplices_exterior = H.alpha_complex.get_alpha_complex_lines(H.alpha, type='exterior')
ac_simplices_interior = H.alpha_complex.get_alpha_complex_lines(H.alpha, type='interior')
ac_simplices_regular = H.alpha_complex.get_alpha_complex_lines(H.alpha, type='regular')
ac_simplices_singular = H.alpha_complex.get_alpha_complex_lines(H.alpha, type='singular')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)

for simp in ac_simplices_all:
    ax.plot(locdata.coordinates[simp, 0], locdata.coordinates[simp, 1], '-b')
for simp in ac_simplices_interior:
    ax.plot(locdata.coordinates[simp, 0], locdata.coordinates[simp, 1], '--g')
for simp in ac_simplices_regular:
    ax.plot(locdata.coordinates[simp, 0], locdata.coordinates[simp, 1], '--r')
for simp in ac_simplices_singular:
    ax.plot(locdata.coordinates[simp, 0], locdata.coordinates[simp, 1], '--y')

locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
plt.show()

Often the *regular* representation is good enough.

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
for simp in ac_simplices_regular:
    ax.plot(locdata.coordinates[simp, 0], locdata.coordinates[simp, 1], '-r')
locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
plt.show()

You can get the connected components as list of `Region`.

In [ ]:
H.connected_components

In [ ]:
connected_component_0 = H.connected_components[0]

print('dimension: ', connected_component_0.dimension)
print('region_measure: ', connected_component_0.region_measure)
print('subregion_measure: ', connected_component_0.subregion_measure)

The alpha shape for a smaller alpha can have multiple connected components.

In [ ]:
H = lc.AlphaShape(5, locdata.coordinates)

print('dimension: ', H.dimension)
# print('vertex_indices: ', H.vertex_indices)
# print('vertices: ', H.vertices)
print('region_measure: ', H.region_measure)
print('subregion_measure: ', H.subregion_measure)
print('points in alpha shape: ', H.n_points_alpha_shape)
print('points in alpha shape relative to all points: ', H.n_points_alpha_shape_rel)
print('points on boundary: ', H.n_points_on_boundary)
print('points on boundary relative to all points: ', H.n_points_on_boundary_rel)

In [ ]:
ac_simplices_all = H.alpha_complex.get_alpha_complex_lines(H.alpha, type='all')
ac_simplices_exterior = H.alpha_complex.get_alpha_complex_lines(H.alpha, type='exterior')
ac_simplices_interior = H.alpha_complex.get_alpha_complex_lines(H.alpha, type='interior')
ac_simplices_regular = H.alpha_complex.get_alpha_complex_lines(H.alpha, type='regular')
ac_simplices_singular = H.alpha_complex.get_alpha_complex_lines(H.alpha, type='singular')

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)

for simp in ac_simplices_all:
    ax.plot(locdata.coordinates[simp, 0], locdata.coordinates[simp, 1], '-b')
for simp in ac_simplices_interior:
    ax.plot(locdata.coordinates[simp, 0], locdata.coordinates[simp, 1], '--g')
for simp in ac_simplices_regular:
    ax.plot(locdata.coordinates[simp, 0], locdata.coordinates[simp, 1], '--r')
for simp in ac_simplices_singular:
    ax.plot(locdata.coordinates[simp, 0], locdata.coordinates[simp, 1], '--y')

locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
plt.show()

The *regular* representation:

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=1)
for simp in ac_simplices_regular:
    ax.plot(locdata.coordinates[simp, 0], locdata.coordinates[simp, 1], '-r')
locdata.data.plot.scatter(x='position_x', y='position_y', ax=ax, color='Blue', label='locdata')
plt.show()

The connected components:

In [ ]:
H.connected_components

In [ ]:
connected_component_0 = H.connected_components[0]

print('dimension: ', connected_component_0.dimension)
print('region_measure: ', connected_component_0.region_measure)
print('subregion_measure: ', connected_component_0.subregion_measure)